In [9]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import time
from tqdm import tqdm

In [10]:
df = pd.read_csv('C:/MachineLearning/Financial_chatbot/Transformers/combined_output.csv')

question = df['Question']
answer = df['Answer']

print("정규 표현식 적용 전 데이터 프레임")
print(df[20:30])

def regex_column(columnList):
  if not isinstance(columnList, str):                                                   # Nan을 빈 문자열로 대체
      return ''
  columnList = re.sub(r'\S+@\S+\.\S+', '', columnList)                                  # 이메일 삭제
  columnList = columnList.replace('\n', '')                                             # 개행 삭제
  columnList = re.sub(r'\[.*?\]|\{.*?\}|\(.*?\)', '', columnList)                       # 소,중,대괄호 내용 삭제
  columnList = re.sub(r'[^가-힣a-zA-Z0-9\u4e00-\u9fff\s.,!?\'\"~]', ' ', columnList)    # 이상한 특수문자 삭제
  columnList = re.sub(r'\s+', ' ', columnList).strip()                                  # 양 끝 공백 삭제
  return columnList

question = question.apply(regex_column)
answer = answer.apply(regex_column)

df = pd.DataFrame({
    'Question':question,
    'Answer':answer
})

print("\n정규 표현식 적용 후 데이터 프레임")
print(df[20:30])

정규 표현식 적용 전 데이터 프레임
                                             Question  \
20  영웅문 hts 0156 조건검색실시간여기에 뜨는 종목을 더블클릭으로 차트랑 8282...   
21  입술 모델을 하고 싶은데 평가해주세욤 어떤지 평가 부탁드립니다ㅜㅜ… 입꼬리는 올라가...   
22  주식계좌에 대해 궁금한 점 질문합니다! 안녕하세요. 이번에 회사동료들과 주식 얘기하...   
23  증권계좌를 만들 때 필요한게 뭐가 있는지 궁금해요! 증권계좌를 이번에 비대면으로 만...   
24  주식  질문 약세 잉태형을 설명하는 글을 읽다 궁금증이 들어서요양봉에 종가가 위에 ...   
25  자산이 얼마인지 궁금해요 주식초보자입니다.제가쓸수있는돈이 얼마남았는지 아시는분은 알...   
26  주식  질문 약세 잉태형을 설명하는 글을 읽다 궁금증이 들어서요양봉에 종가가 위에 ...   
27  주식 문의 제가 주식을 시작한지 얼마 안된 주린이인데요막 몇천개되는 종목등을 일일히...   
28  주식을 사고 목록에 안떠요 낮은 금액으로 샀는데 안될경우 다시 들어오나요?아님 정상...   
29  주식 질문이요 보유주식 팔아도 해당 주식 배당금 수령 가능한가요?최근에 A 주식 1...   

                                               Answer  
20  질문자님 안녕하세요.​영웅문 hts0156 조건검색실시간여기에 뜨는 종목을 더블클릭...  
21  질문자님 안녕하세요.​입술 모델을 하고 싶은데 평가해주세욤어떤지 평가 부탁드립니다ㅜ...  
22  질문자님 안녕하세요.​주식계좌에 대해 궁금한 점 질문합니다!안녕하세요. 이번에 회사...  
23  질문자님 안녕하세요.​증권계좌를 만들 때 필요한게 뭐가 있는지 궁금해요!증권계좌를 ...  
24  질문자님 안녕하세요.​주식 질문약세 잉태형을 설명하는 글을 읽다 궁금증이 들어서요양...  
25  질문자님 안녕하세요.​

In [11]:
# Question과 Answer의 길이 계산
df['Question_length'] = df['Question'].apply(len)
df['Answer_length'] = df['Answer'].apply(len)

# 길이 통계 출력
question_mean = df['Question_length'].mean()
question_min = df['Question_length'].min()
question_max = df['Question_length'].max()

answer_mean = df['Answer_length'].mean()
answer_min = df['Answer_length'].min()
answer_max = df['Answer_length'].max()

print("Question 길이 통계:")
print(f"평균: {question_mean}, 최소: {question_min}, 최대: {question_max}")

print("Answer 길이 통계:")
print(f"평균: {answer_mean}, 최소: {answer_min}, 최대: {answer_max}")

Question 길이 통계:
평균: 138.50321925340182, 최소: 0, 최대: 10642
Answer 길이 통계:
평균: 964.6832288230208, 최소: 0, 최대: 52821


In [12]:
question_min_length = 5  # Question의 최소 길이
question_max_length = 200  # Question의 최대 길이
answer_min_length = 10    # Answer의 최소 길이
answer_max_length = 800     # Answer의 최대 길이

# 조건에 따라 필터링
filtered_df = df[
    (df['Question'].apply(len) >= question_min_length) &
    (df['Question'].apply(len) <= question_max_length) &
    (df['Answer'].apply(len) >= answer_min_length) &
    (df['Answer'].apply(len) <= answer_max_length)
]

print("필터링된 데이터프레임:")
print(filtered_df)

필터링된 데이터프레임:
                                                 Question  \
0       20영업일 계좌개설 제한 2021.06.09 에 계좌개설했는데 20영업일 계좌개설 ...   
1       주식 주문관련금액 주문관련금액에 8만원이 있는데 출금은 못하나고 주문만 할수있는건가요??   
2       주식 출금가능금액 질문이요... 제가 학생이라 10만원정도가 있는데 그중 8만정도가...   
3       안녕하세요, 가치 투자의 진정한 뜻이... 안녕하세요,가치 투자의 진정한 뜻이 궁금...   
5       주식 출금가능금액 제가 저번주에 주식하나를 매도했는데 주말제외해도 매도한지 4일이 ...   
...                                                   ...   
119118  음주운전 결격기간이 지나도 특별교통안전교육 받아야하나요 음주운전 결격기간이 23년 ...   
119120                                바베이도스 축구 잘해요? 잘하나요?   
119121  그런가요???? 제가 지병이 있어서 대학병원을 다니는데신경과 교수들은 원래이러케 단...   
119123  고1 수학 공부 지금 예비고1이라 고1수학 선행하고 있는데 지금 다항식의 연산 기본...   
119125  군대 질문 요번에 3월에 가는 해병대 동반입대에 지원했는데 만약에 1차합격을 한 상...   

                                                   Answer  Question_length  \
0       질문자님 안녕하세요. 20영업일 계좌개설 제한2021.06.09 에 계좌개설했는데 ...               58   
1       질문자님 안녕하세요. 주식 주문관련금액주문관련금액에 8만원이 있는데 출금은 못하나고...               49   
2       질문자님 안녕하세요. 

In [13]:
print(df)
df = filtered_df.dropna()
print(len(df))
df = df.iloc[:,:2]
print(df)

                                                 Question  \
0       20영업일 계좌개설 제한 2021.06.09 에 계좌개설했는데 20영업일 계좌개설 ...   
1       주식 주문관련금액 주문관련금액에 8만원이 있는데 출금은 못하나고 주문만 할수있는건가요??   
2       주식 출금가능금액 질문이요... 제가 학생이라 10만원정도가 있는데 그중 8만정도가...   
3       안녕하세요, 가치 투자의 진정한 뜻이... 안녕하세요,가치 투자의 진정한 뜻이 궁금...   
4       체결 후 입금이 안 되네요~ 주식을 매도 했습니다 .보유 주식에도 빠져 나갔구요, ...   
...                                                   ...   
119122  울분 화병은 가만있다가 과거 안좋은기억 때문에 울화가치밀어서 가슴이 답답해서 소리지...   
119123  고1 수학 공부 지금 예비고1이라 고1수학 선행하고 있는데 지금 다항식의 연산 기본...   
119124  상가 임대차에 관한 문의드립니다 기존매장 인수를하는데 기존 상가임대차계약이 25년 ...   
119125  군대 질문 요번에 3월에 가는 해병대 동반입대에 지원했는데 만약에 1차합격을 한 상...   
119126   광주 영구 눈썹문신 샵 광주광역시 반영구 눈썹 말구 영구 눈썹 문신 샵들 좀 알려주세용   

                                                   Answer  Question_length  \
0       질문자님 안녕하세요. 20영업일 계좌개설 제한2021.06.09 에 계좌개설했는데 ...               58   
1       질문자님 안녕하세요. 주식 주문관련금액주문관련금액에 8만원이 있는데 출금은 못하나고...               49   
2       질문자님 안녕하세요. 주식 출금가능금액 질문이

In [15]:
questions = []
for question in df['Question']:
    questions.append(question)
answers = []
for answer in df['Answer']:
    answers.append(answer)

start_time = time.time()

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size = 2**14
)

end_time = time.time()
total_time = end_time - start_time

# 초를 시, 분, 초로 변환
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = total_time % 60

print(f"토크나이저 총 소요 시간: {hours}시간 {minutes}분 {seconds:.2f}초")
tokenizer.save_to_file('tokenizer')

토크나이저 총 소요 시간: 0시간 16분 26.54초


In [18]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file('tokenizer')

start_time = time.time()
beforeQuestion = questions[99]
beforeAnswer = answers[99]

START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size+1]
VOCAB_SIZE = tokenizer.vocab_size + 2

SEN_MAX_LENGTH = 800
ABS_MAX_LENGTH = 200

def tokenized_and_padding(questions, answers):
  tokenized_question, tokenized_answer = [], []
  
  for (question, answer) in zip(questions, answers):
    question = START_TOKEN + tokenizer.encode(question) + END_TOKEN
    answer = START_TOKEN + tokenizer.encode(answer) + END_TOKEN
    
    tokenized_question.append(question)
    tokenized_answer.append(answer)
    
  tokenized_question = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_question, maxlen=SEN_MAX_LENGTH, padding='post'
    )
  tokenized_answer = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_answer, maxlen=ABS_MAX_LENGTH, padding='post'
  )
  
  return tokenized_question, tokenized_answer

questions, answers = tokenized_and_padding(questions, answers)

print(f'question 원본 : {beforeQuestion}')
print(f'\nsentence 토큰화 및 패딩 결과 : {questions[99]}')
print(f'\nabs 원본 : {beforeAnswer}')
print(f'\nabs 토큰화 및 패딩 결과 : {answers[99]}')

end_time = time.time()
total_time = end_time - start_time
minutes, seconds = divmod(total_time, 60)

print(f"\n총 소요 시간: {int(minutes)}분 {seconds:.2f}초")

TypeError: Expected binary or unicode string, got array([16393, 16130,  1564,  3036,  5840, 16183,  5292, 16183,  4458,
           8,  2180,   362, 16130,  1564,  3036,   654,  6331,    78,
          66, 16394,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0

In [20]:
import pickle

with open('questions.pkl', 'wb') as f:
    pickle.dump(questions, f)

with open('answers.pkl', 'wb') as f:
    pickle.dump(answers, f)

print('피클 저장 완료!')

피클 저장 완료!
